<a href="https://colab.research.google.com/github/jnpos/group8_ECG/blob/Classifier/ECG_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/jveenland/tm10007_ml.git

fatal: destination path 'tm10007_ml' already exists and is not an empty directory.


In [7]:
# General packages
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import datasets as ds
from sklearn import metrics
import zipfile
import os
import pandas as pd

# Metrics
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import r2_score

# Classifiers
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn import preprocessing

In [8]:
with zipfile.ZipFile('/content/tm10007_ml/ecg/ecg_data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/tm10007_ml/ecg')

data = pd.read_csv('/content/tm10007_ml/ecg/ecg_data.csv', index_col=0)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')



The number of samples: 827
The number of columns: 9001


**Split input, output, test and train data:**

In [9]:
# Find column with label 
bool_cols = [col for col in data 
             if np.isin(data[col].dropna().unique(), [0, 1]).all()]
loc_label = data.columns.get_loc('label')

# Determine data and output 
y = data['label']
x = pd.DataFrame()
x = data.drop(data.columns[loc_label],axis=1)

# Split test and trainingsdata 
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y,test_size=0.25,random_state=0,stratify=y)



**Scaling:**

In [10]:
# SCALEN
scaler = preprocessing.RobustScaler()
scaler.fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train))

# Niet fitten op test, alleen toepassen


**Checking if data is normally distributed:**


In [11]:
# Checking if data is normally distributed  

from scipy.stats import shapiro  
shapiro(x_train)

amount_normallydistributed = 0
for column in x_train.columns:
  result = shapiro(x_train[column])
  normallydistributed = result.pvalue > 0.05
  amount_normallydistributed += normallydistributed 
  
print(amount_normallydistributed, "features are normally distributed features")


/usr/local/lib/python3.9/dist-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


0 features are normally distributed features


**PCA:**

In [12]:
# PCA
n_samples = len(x_train)
n_features = len(x_train.columns)
n_features = min((n_samples-1), n_features)
print(n_features)

p = PCA(n_components=n_features)
p = p.fit(x_train)
x_pca = p.transform(x_train)

print(f'result pca: {x_pca.shape}')

619
result pca: (620, 619)


**Feature selection:**

In [13]:
# Feature selection

# Univariate & f_classif
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(f_classif,k="all")
selector.fit(x_pca,y_train)
scores = -np.log10(selector.pvalues_)
scores /= scores.max()

dataframe = pd.DataFrame()
dataframe = selector.pvalues_

print(len(selector.pvalues_[selector.pvalues_<0.05]))

fs = SelectKBest(score_func=f_classif, k=len(selector.pvalues_[selector.pvalues_<0.05]))
X_univariate = fs.fit_transform(x_pca, y_train)
print(f'result univariate: {X_univariate.shape}')

# Elastic net --> is dit wel een goede fs?
from sklearn.linear_model import ElasticNet
from sklearn.feature_selection import SelectFromModel

ENreg = ElasticNet(alpha=1, l1_ratio=0.5).fit(x_pca,y_train)
model_ENreg = SelectFromModel(ENreg, prefit=True)
x_ENreg = model_ENreg.transform(x_pca)

print(f'result Elastic Net: {x_ENreg.shape}')

#x_ENreg = pd.DataFrame(x_ENreg)


36
result univariate: (620, 36)
result Elastic Net: (620, 13)


**Optimize hyperparameters of all classifiers:**

LSVM

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics import f1_score, make_scorer
from sklearn.svm import LinearSVC
from sklearn.metrics import fbeta_score

l_svm = LinearSVC()
#logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,
                              #random_state=0)
distributions = dict(penalty=['l2', 'l1'],
                     loss = ['hinge','squared_hinge'],                                       
                     dual=[False, False],
                     tol = np.geomspace(1e-7, 1e2, num=10),
                     C = np.geomspace(1e-5, 1e5, num=11),
                     fit_intercept=[True, True],
                     intercept_scaling = range(1,10),
                     class_weight = ['balanced','balanced'])
#y_pred = clf.predict(x_train)
#l_svm.fit(x_train, y_train)


#y_pred = l_svm.fit(x_train, y_train).predict(x_train)

clf = RandomizedSearchCV(l_svm, distributions, random_state=0, error_score=0.0, scoring=make_scorer(fbeta_score, beta=2)) #dual = False als n_samples > n_features 
search = clf.fit(x_train, y_train)
print(search.best_params_)




[LibLinear][LibLinear][LibLinear][LibLinear]

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[LibLinear][LibLinear]

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[LibLinear]

...

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to 0.0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/svm/_classes.py", line 274, in fit
    self.coef_, self.intercept_, n_iter_ = _fit_liblinear(
  File "/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py", line 1223, in _fit_liblinear
    solver_type = _get_liblinear_solver_type(multi_class, penalty, los

[LibLinear]{'verbose': 2, 'tol': 0.0001, 'penalty': 'l1', 'loss': 'squared_hinge', 'intercept_scaling': 2, 'fit_intercept': True, 'dual': False, 'class_weight': 'balanced', 'C': 0.01}


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


KNN

In [20]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

distributions = dict(n_neighbors=range(1,10),
                     weights=['uniform','distance', None],
                     algorithm=['auto','auto'],
                     leaf_size=range(0,100),
                     p=[1,2])

clf = RandomizedSearchCV(knn, distributions, random_state=0, error_score=0.0, scoring=make_scorer(fbeta_score, beta=2)) 
search = clf.fit(x_train, y_train)
print(search.best_params_)



{'weights': 'distance', 'p': 1, 'n_neighbors': 2, 'leaf_size': 46, 'algorithm': 'auto'}


DT

In [34]:
from sklearn import tree
import numpy as np

tree = tree.DecisionTreeClassifier()

distributions = dict(criterion =['gini','entropy','log_loss'],
                     splitter=['best','random'],
                     min_samples_split=range(2,10),
                     min_samples_leaf=range(1,10),
                     min_weight_fraction_leaf = np.linspace(0, 0.5, 25),
                     max_features=['sqrt','log2',None],
                     class_weight=['balanced','balanced'])

clf = RandomizedSearchCV(tree, distributions, random_state=0, error_score=0.0, scoring=make_scorer(fbeta_score, beta=2)) 
search = clf.fit(x_train, y_train)
print(search.best_params_)

{'splitter': 'random', 'min_weight_fraction_leaf': 0.125, 'min_samples_split': 6, 'min_samples_leaf': 8, 'max_features': None, 'criterion': 'gini', 'class_weight': 'balanced'}


SVC

In [39]:
from sklearn import svm

svm = svm.SVC()

distributions = dict(C = np.geomspace(1e-5, 1e5, num=11),
                     kernel=['linear','poly','rbf','sigmoid'],
                     degree = range(0,5),
                     gamma=['scale','auto'],
                     coef0 = range(-5,5),
                     shrinking = [True,False],
                     probability = [True, False],
                     tol = np.geomspace(1e-7, 1e2, num=10),
                     cache_size=range(10,400),
                     class_weight = ['balanced','balanced'])

clf = RandomizedSearchCV(svm, distributions, random_state=0, error_score=0.0, scoring=make_scorer(fbeta_score, beta=2)) #dual = False als n_samples > n_features 

search = clf.fit(x_train, y_train)
print(search.best_params_)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]{'verbose': 1, 'tol': 100.0, 'shrinking': False, 'probability': False, 'kernel': 'poly', 'gamma': 'auto', 'degree': 2, 'coef0': 1, 'class_weight': 'balanced', 'cache_size': 150, 'C': 0.01}


RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()

distributions = dict(n_estimators = range(5,1000,5),
                     criterion =['gini','entropy','log_loss'],
                     min_samples_split=range(2,10),
                     min_samples_leaf=range(1,10),
                     min_weight_fraction_leaf = np.linspace(0, 0.5, 25),
                     max_features=['sqrt','log2',None],
                     bootstrap=[True,False],
                     oob_score=[True,False],
                     warm_start=[True,False],
                     class_weight=['balanced','balanced_subsample'])

clf = RandomizedSearchCV(RF, distributions, random_state=0, error_score=0.0, scoring=make_scorer(fbeta_score, beta=2)) 
search = clf.fit(x_train, y_train)
print(search.best_params_)

/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:777: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(
/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_forest.py:777: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the cl